### CNN on CIFR Assignment:

1.  Please visit this link to access the state-of-art DenseNet code for reference - DenseNet - cifar10 notebook link
2.  You need to create a copy of this and "retrain" this model to achieve 90+ test accuracy. 
3.  You cannot use Dense Layers (also called fully connected layers), or DropOut.
4.  You MUST use Image Augmentation Techniques.
5.  You cannot use an already trained model as a beginning points, you have to initilize as your own
6.  You cannot run the program for more than 300 Epochs, and it should be clear from your log, that you have only used 300 Epochs
7.  You cannot use test images for training the model.
8.  You cannot change the general architecture of DenseNet (which means you must use Dense Block, Transition and Output blocks as mentioned in the code)
9.  You are free to change Convolution types (e.g. from 3x3 normal convolution to Depthwise Separable, etc)
10. You cannot have more than 1 Million parameters in total
11. You are free to move the code from Keras to Tensorflow, Pytorch, MXNET etc. 
12. You can use any optimization algorithm you need. 
13. You can checkpoint your model and retrain the model from that checkpoint so that no need of training the model from first if you lost at any epoch while training. You can directly load that model and Train from that epoch. 

In [36]:
# import keras
# from keras.datasets import cifar10
# from keras.models import Model, Sequential
# from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
# from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
# from keras.layers import Concatenate
# from keras.optimizers import Adam
from keras import models, layers
from keras.models import Model
from keras.layers import BatchNormalization, Activation, Flatten
from keras.optimizers import Adam
from keras.datasets import cifar10
import keras
import pandas as pd
import numpy as np

In [11]:
# Hyperparameters
#batch_size = 64
num_classes = 10
l = 6
num_filter = 32
compression = 1

In [12]:
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes) 

In [13]:
X_train = X_train.astype('float32') / 255

X_train.shape

(50000, 32, 32, 3)

In [14]:
X_test = X_test.astype('float32') / 255

X_test.shape

(10000, 32, 32, 3)

In [152]:
# Dense Block
def denseblock(input, num_filter = 12):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
      
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

## transition Blosck
def transition(input, num_filter = 12):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    c=layers.Conv2D(10, (1, 1), padding='valid')(AvgPooling)

    avg=layers.GlobalAveragePooling2D()(c)
    
    output=layers.Activation('softmax')(avg)
    
    return output

In [153]:
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter)
First_Transition = transition(First_Block, num_filter)

Second_Block = denseblock(First_Transition, num_filter)
Second_Transition = transition(Second_Block, num_filter)

Third_Block = denseblock(Second_Transition, num_filter)
Third_Transition = transition(Third_Block, num_filter)

Last_Block = denseblock(Third_Transition,  num_filter)
output = output_layer(Last_Block)

In [154]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_335 (Conv2D)             (None, 32, 32, 32)   864         input_11[0][0]                   
__________________________________________________________________________________________________
batch_normalization_329 (BatchN (None, 32, 32, 32)   128         conv2d_335[0][0]                 
__________________________________________________________________________________________________
activation_335 (Activation)     (None, 32, 32, 32)   0           batch_normalization_329[0][0]    
__________________________________________________________________________________________________
conv2d_336

In [155]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [156]:
from keras.callbacks import ModelCheckpoint

In [157]:
filepath="best_model.hdf5"
checkpoint=ModelCheckpoint(filepath,monitor='val_acc',verbose=1,save_best_only=True,mode='max')
callbacks_list =[checkpoint]

### Image Augmentation

In [158]:
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/

# https://medium.com/@arindambaidya168/https-medium-com-arindambaidya168-using-keras-imagedatagenerator-b94a87cdefad

In [5]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rotation_range=30,
                             zoom_range=0.1,
                             width_shift_range=0.15, 
                             height_shift_range=0.15, 
                             horizontal_flip=True
                            )

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
datagen.fit(X_train)

# fit the model on the batches generated by datagen.flow()
history=model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
                        steps_per_epoch=X_train.shape[0] // 64,
                        validation_data=(X_test, y_test),
                        epochs=25, verbose=1,callbacks=callbacks_list)

Epoch 1/25
781/781 [==============================] - 7165s 9s/step - loss: 1.6435 - acc: 0.3934 - val_loss: 2.3846 - val_acc: 0.3026

Epoch 00001: val_acc improved from -inf to 0.30260, saving model to best_model.hdf5
Epoch 2/25
781/781 [==============================] - 7254s 9s/step - loss: 1.3133 - acc: 0.5272 - val_loss: 1.2857 - val_acc: 0.5548

Epoch 00002: val_acc improved from 0.30260 to 0.55480, saving model to best_model.hdf5
Epoch 3/25
781/781 [==============================] - 7230s 9s/step - loss: 1.1488 - acc: 0.5903 - val_loss: 1.4544 - val_acc: 0.5106

Epoch 00003: val_acc did not improve from 0.55480
Epoch 4/25
781/781 [==============================] - 7245s 9s/step - loss: 1.0399 - acc: 0.6317 - val_loss: 0.9488 - val_acc: 0.6618

Epoch 00004: val_acc improved from 0.55480 to 0.66180, saving model to best_model.hdf5
Epoch 5/25
781/781 [==============================] - 7159s 9s/step - loss: 0.9630 - acc: 0.6600 - val_loss: 0.9395 - val_acc: 0.6792

Epoch 00005: val_

In [22]:
# training from 21st epoch as laptop was frozen

In [23]:
from keras import models    
model = models.load_model('best_model.hdf5')

In [24]:
from keras.callbacks import ModelCheckpoint

filepath="best_model_r2.hdf5"
checkpoint=ModelCheckpoint(filepath,monitor='val_acc',verbose=1,save_best_only=True,mode='max')
callbacks_list =[checkpoint]

In [26]:
datagen.fit(X_train)

history=model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
                        steps_per_epoch=X_train.shape[0] // 64,
                        validation_data=(X_test, y_test),initial_epoch=20,
                        epochs=30, verbose=1,callbacks=callbacks_list)

Epoch 21/30
781/781 [==============================] - 7145s 9s/step - loss: 0.5239 - acc: 0.8199 - val_loss: 0.5421 - val_acc: 0.8173

Epoch 00021: val_acc improved from -inf to 0.81730, saving model to best_model_r2.hdf5
Epoch 22/30
781/781 [==============================] - 7452s 10s/step - loss: 0.5266 - acc: 0.8179 - val_loss: 0.7297 - val_acc: 0.7637

Epoch 00022: val_acc did not improve from 0.81730
Epoch 23/30
781/781 [==============================] - 7433s 10s/step - loss: 0.5037 - acc: 0.8254 - val_loss: 0.6858 - val_acc: 0.7783

Epoch 00023: val_acc did not improve from 0.81730
Epoch 24/30
781/781 [==============================] - 7820s 10s/step - loss: 0.4921 - acc: 0.8298 - val_loss: 0.6768 - val_acc: 0.7793

Epoch 00024: val_acc did not improve from 0.81730
Epoch 25/30
781/781 [==============================] - 8927s 11s/step - loss: 0.4948 - acc: 0.8304 - val_loss: 0.6798 - val_acc: 0.7927

Epoch 00025: val_acc did not improve from 0.81730
Epoch 26/30
781/781 [========

In [2]:
from keras import models    
model = models.load_model('best_model_r2.hdf5')

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [3]:
from keras.callbacks import ModelCheckpoint

filepath="best_model_r3.hdf5"
checkpoint=ModelCheckpoint(filepath,monitor='val_acc',verbose=1,save_best_only=True,mode='max')
callbacks_list =[checkpoint]

In [15]:
history=model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
                        steps_per_epoch=X_train.shape[0] // 64,
                        validation_data=(X_test, y_test),initial_epoch=30,
                        epochs=40, verbose=1,callbacks=callbacks_list)

Epoch 31/40
781/781 [==============================] - 7972s 10s/step - loss: 0.4449 - acc: 0.8478 - val_loss: 0.4478 - val_acc: 0.8471

Epoch 00031: val_acc improved from -inf to 0.84710, saving model to best_model_r3.hdf5
Epoch 32/40
781/781 [==============================] - 7900s 10s/step - loss: 0.4403 - acc: 0.8476 - val_loss: 0.4321 - val_acc: 0.8550

Epoch 00032: val_acc improved from 0.84710 to 0.85500, saving model to best_model_r3.hdf5
Epoch 33/40
781/781 [==============================] - 8036s 10s/step - loss: 0.4174 - acc: 0.8568 - val_loss: 0.5456 - val_acc: 0.8284

Epoch 00033: val_acc did not improve from 0.85500
Epoch 34/40
781/781 [==============================] - 7304s 9s/step - loss: 0.4423 - acc: 0.8473 - val_loss: 0.5238 - val_acc: 0.8305

Epoch 00034: val_acc did not improve from 0.85500
Epoch 35/40
781/781 [==============================] - 9818s 13s/step - loss: 0.4165 - acc: 0.8550 - val_loss: 0.5210 - val_acc: 0.8325

Epoch 00035: val_acc did not improve fr

In [16]:
history=model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
                        steps_per_epoch=X_train.shape[0] // 64,
                        validation_data=(X_test, y_test),initial_epoch=40,
                        epochs=45, verbose=1,callbacks=callbacks_list)

Epoch 41/45
781/781 [==============================] - 7039s 9s/step - loss: 0.3832 - acc: 0.8677 - val_loss: 0.7079 - val_acc: 0.7862

Epoch 00041: val_acc did not improve from 0.85510
Epoch 42/45
781/781 [==============================] - 7041s 9s/step - loss: 0.3794 - acc: 0.8668 - val_loss: 0.4529 - val_acc: 0.8535

Epoch 00042: val_acc did not improve from 0.85510
Epoch 43/45
781/781 [==============================] - 8605s 11s/step - loss: 0.3727 - acc: 0.8716 - val_loss: 0.4276 - val_acc: 0.8571

Epoch 00043: val_acc improved from 0.85510 to 0.85710, saving model to best_model_r3.hdf5
Epoch 44/45
781/781 [==============================] - 7732s 10s/step - loss: 0.3727 - acc: 0.8707 - val_loss: 0.4130 - val_acc: 0.8646

Epoch 00044: val_acc improved from 0.85710 to 0.86460, saving model to best_model_r3.hdf5
Epoch 45/45
781/781 [==============================] - 7060s 9s/step - loss: 0.3565 - acc: 0.8773 - val_loss: 0.4819 - val_acc: 0.8437

Epoch 00045: val_acc did not improve f

In [17]:
history=model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
                        steps_per_epoch=X_train.shape[0] // 64,
                        validation_data=(X_test, y_test),initial_epoch=45,
                        epochs=50, verbose=1,callbacks=callbacks_list)

Epoch 46/50
781/781 [==============================] - 7031s 9s/step - loss: 0.3610 - acc: 0.8754 - val_loss: 0.4859 - val_acc: 0.8376

Epoch 00046: val_acc did not improve from 0.86460
Epoch 47/50
781/781 [==============================] - 7041s 9s/step - loss: 0.3566 - acc: 0.8763 - val_loss: 0.4703 - val_acc: 0.8504

Epoch 00047: val_acc did not improve from 0.86460
Epoch 48/50
781/781 [==============================] - 7025s 9s/step - loss: 0.3460 - acc: 0.8806 - val_loss: 0.4636 - val_acc: 0.8507

Epoch 00048: val_acc did not improve from 0.86460
Epoch 49/50
781/781 [==============================] - 7160s 9s/step - loss: 0.3531 - acc: 0.8770 - val_loss: 0.4924 - val_acc: 0.8372

Epoch 00049: val_acc did not improve from 0.86460
Epoch 50/50
781/781 [==============================] - 7041s 9s/step - loss: 0.3465 - acc: 0.8787 - val_loss: 0.4686 - val_acc: 0.8475

Epoch 00050: val_acc did not improve from 0.86460


In [18]:
history=model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
                        steps_per_epoch=X_train.shape[0] // 64,
                        validation_data=(X_test, y_test),initial_epoch=50,
                        epochs=60, verbose=1,callbacks=callbacks_list)

Epoch 51/60
781/781 [==============================] - 7030s 9s/step - loss: 0.3279 - acc: 0.8874 - val_loss: 0.4154 - val_acc: 0.8639

Epoch 00051: val_acc did not improve from 0.86460
Epoch 52/60
781/781 [==============================] - 7008s 9s/step - loss: 0.3430 - acc: 0.8818 - val_loss: 0.3911 - val_acc: 0.8710

Epoch 00052: val_acc improved from 0.86460 to 0.87100, saving model to best_model_r3.hdf5
Epoch 53/60
781/781 [==============================] - 7009s 9s/step - loss: 0.3316 - acc: 0.8855 - val_loss: 0.4665 - val_acc: 0.8517

Epoch 00053: val_acc did not improve from 0.87100
Epoch 54/60
781/781 [==============================] - 7011s 9s/step - loss: 0.3328 - acc: 0.8844 - val_loss: 0.4882 - val_acc: 0.8448

Epoch 00054: val_acc did not improve from 0.87100
Epoch 55/60
781/781 [==============================] - 7009s 9s/step - loss: 0.3172 - acc: 0.8879 - val_loss: 0.4811 - val_acc: 0.8421

Epoch 00055: val_acc did not improve from 0.87100
Epoch 56/60
781/781 [=========

In [19]:
history=model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
                        steps_per_epoch=X_train.shape[0] // 64,
                        validation_data=(X_test, y_test),initial_epoch=60,
                        epochs=75, verbose=1,callbacks=callbacks_list)

Epoch 61/75
781/781 [==============================] - 6984s 9s/step - loss: 0.2997 - acc: 0.8966 - val_loss: 0.4515 - val_acc: 0.8562

Epoch 00061: val_acc did not improve from 0.87580
Epoch 62/75
781/781 [==============================] - 7075s 9s/step - loss: 0.3074 - acc: 0.8953 - val_loss: 0.4174 - val_acc: 0.8682

Epoch 00062: val_acc did not improve from 0.87580
Epoch 63/75
781/781 [==============================] - 6977s 9s/step - loss: 0.2914 - acc: 0.8994 - val_loss: 0.4514 - val_acc: 0.8603

Epoch 00063: val_acc did not improve from 0.87580
Epoch 64/75
781/781 [==============================] - 6978s 9s/step - loss: 0.3007 - acc: 0.8932 - val_loss: 0.4122 - val_acc: 0.8678

Epoch 00064: val_acc did not improve from 0.87580
Epoch 65/75
781/781 [==============================] - 6973s 9s/step - loss: 0.2927 - acc: 0.8968 - val_loss: 0.4490 - val_acc: 0.8585

Epoch 00065: val_acc did not improve from 0.87580
Epoch 66/75
781/781 [==============================] - 6980s 9s/step -

In [20]:
history=model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
                        steps_per_epoch=X_train.shape[0] // 64,
                        validation_data=(X_test, y_test),initial_epoch=75,
                        epochs=100, verbose=1,callbacks=callbacks_list)

Epoch 76/100
781/781 [==============================] - 9327s 12s/step - loss: 0.2564 - acc: 0.9107 - val_loss: 0.4065 - val_acc: 0.8749

Epoch 00076: val_acc did not improve from 0.88780
Epoch 77/100
781/781 [==============================] - 7234s 9s/step - loss: 0.2820 - acc: 0.9006 - val_loss: 0.3394 - val_acc: 0.8886

Epoch 00077: val_acc improved from 0.88780 to 0.88860, saving model to best_model_r3.hdf5
Epoch 78/100
781/781 [==============================] - 6954s 9s/step - loss: 0.2556 - acc: 0.9107 - val_loss: 0.3949 - val_acc: 0.8769

Epoch 00078: val_acc did not improve from 0.88860
Epoch 79/100
781/781 [==============================] - 7318s 9s/step - loss: 0.2636 - acc: 0.9102 - val_loss: 0.3464 - val_acc: 0.8875

Epoch 00079: val_acc did not improve from 0.88860
Epoch 80/100
781/781 [==============================] - 9067s 12s/step - loss: 0.2565 - acc: 0.9109 - val_loss: 0.3577 - val_acc: 0.8852

Epoch 00080: val_acc did not improve from 0.88860
Epoch 81/100
781/781 [=

In [21]:
history=model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
                        steps_per_epoch=X_train.shape[0] // 64,
                        validation_data=(X_test, y_test),initial_epoch=100,
                        epochs=110, verbose=1,callbacks=callbacks_list)

Epoch 101/110
781/781 [==============================] - 7854s 10s/step - loss: 0.2070 - acc: 0.9281 - val_loss: 0.3600 - val_acc: 0.8895

Epoch 00101: val_acc did not improve from 0.89540
Epoch 102/110
781/781 [==============================] - 7451s 10s/step - loss: 0.2288 - acc: 0.9207 - val_loss: 0.3849 - val_acc: 0.8841

Epoch 00102: val_acc did not improve from 0.89540
Epoch 103/110
781/781 [==============================] - 7383s 9s/step - loss: 0.2245 - acc: 0.9205 - val_loss: 0.3560 - val_acc: 0.8889

Epoch 00103: val_acc did not improve from 0.89540
Epoch 104/110
781/781 [==============================] - 8090s 10s/step - loss: 0.2169 - acc: 0.9233 - val_loss: 0.3737 - val_acc: 0.8849

Epoch 00104: val_acc did not improve from 0.89540
Epoch 105/110
781/781 [==============================] - 7275s 9s/step - loss: 0.2098 - acc: 0.9263 - val_loss: 0.3889 - val_acc: 0.8848

Epoch 00105: val_acc did not improve from 0.89540
Epoch 106/110
781/781 [==============================] - 

In [28]:
model = models.load_model('best_model_r3.hdf5')

In [29]:
score = model.evaluate(X_test, y_test, verbose=1)

10000/10000 [==============================] - 1393s 139ms/step


In [30]:
# score

score

[0.3211979459583759, 0.9006]

In [31]:
classes = {
    0: 'airplane',
    1: 'automobile',
    2: 'bird',
    3: 'cat',
    4: 'deer',
    5: 'dog',
    6: 'frog',
    7: 'horse',
    8: 'ship',
    9: 'truck',
    
}

In [32]:
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([classes[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([classes[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [37]:
# confusion matrix

print(confusion_matrix(y_test, model.predict(X_test)))

Pred        airplane  automobile  bird  cat  deer  dog  frog  horse  ship  \
True                                                                        
airplane         933           6    12    7     2    0     7      4    17   
automobile         2         982     1    0     0    0     1      0     2   
bird              32           3   854   15    25    9    45      8     4   
cat                7          10    35  795    20   63    47      7     4   
deer               8           2    20   11   905   10    28     14     1   
dog                4           2    22  100    25  800    30     13     0   
frog               3           2    11    7     3    0   968      1     2   
horse              9           3    14   12    16   22     4    917     0   
ship              33          16     4    1     1    0     4      2   926   
truck              7          55     2    0     0    0     2      1     7   

Pred        truck  
True               
airplane       12  
automobile     

### Summary

1. Used image augmentation techniques on the data set to classify the classes.

2. On average each epoch took 2hrs to train(might be due to low GPUs) .

3. On the 109th epoch we have got test accuracy more that 90%.